### Q1:Expected Value

Use Excel file, using Maximum Likelihood estimate the distribution of units of each year. *Tip. Use only Normal and Uniform distribution.*

Example: 
$$Year 1  \sim \mathcal {N}((200k,10k)$$

Consider a company with the forecast for units over a project for the next 5 years.(in excel)

The project has an initial investment of **1.8M**, a fixed cost of **40k**. Price per unit **10** and cost per unit **8**
The risk free of **3\%**

    1) What is the Expected value of the NPV?  
    2) What is the VAR of NPV?

Answer analytically (Expand your steps of expected value and variance)

    3) What is the Expected value of the NPV?  
    4) What is the VAR of NPV?
    5) What distribution does the NPV have? (use Maximum Likehood,Use only Normal and Uniform distribution)
    6) What probability of having NPV over a 1 M? Explain your answer

In [22]:
# Importamos librerias
import pandas as pd
import numpy as np
import scipy.stats as st
import warnings
import matplotlib.pyplot as plt

In [23]:
# Importamos datos
data = pd.read_excel('EXAMEN_P1.xlsx')
data

,Year 1,Year 2,Year 3,Year 4,Year 5
0,342299,210646,267698,304886,50162
1,224626,244301,398344,230886,49800
2,64300,202526,146428,290904,50120
3,127986,326343,286020,273425,49907
4,402335,367261,398079,226830,50080
...,...,...,...,...,...
2995,184015,262116,377285,199294,50198
2996,194969,393200,157290,225308,49912
2997,172223,374372,398071,187111,50033
2998,18656,459320,217392,212004,50120


In [25]:
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

  # Distributions to check
    DISTRIBUTIONS = [        
      st.norm,st.uniform # Distribuciones usadas por el Tip dado
    ]

    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass

    return (best_distribution.name, best_params)

def make_pdf(dist, params, size=10000):
    """Generate distributions's Probability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

In [27]:
years = list(data.columns)

best_fits = [list(best_fit_distribution(data[year])) for year in years]

best_fits

[['norm', (202192.366, 99037.34944922569)],
 ['uniform', (200099.0, 299876.0)],
 ['uniform', (0.0, 400000.0)],
 ['uniform', (175114.0, 149883.0)],
 ['norm', (49998.439333333336, 99.3904236813364)]]

In [49]:
npv_vec.mean(), npv_vec.std()

(-27823.85909718645, 372841.42601437017)

In [50]:
best_fit_distribution(npv_vec)

('norm', (-27823.85909718645, 372841.42601437017))

In [51]:
mill = [n for n in npv_vec if n>1000000]
ans1=len(mill)/len(data)
print(ans1*100,'%')

0.0 %


### <span style="background:#FFFF00">Answer 1 </span>
    1)What is the Expected value of the NPV?
> -27823.85909718645

    2)What is the VAR of NPV? 
> 372841.42601437017

**Answer analytically**
 
     3) What is the Expected value of the NPV?

$ E[x] = - I + \frac{P_{n}-F_{c}}{(1 - r)^{n}} $

$ E[x] = - 1.8M + \frac{P_{1}-F_{c}}{(1 - r)^{1}} + \frac{P_{2}-F_{c}}{(1 - r)^{2}} + \frac{P_{3}-F_{c}}{(1 - r)^{3}} + \frac{P_{4}-F_{c}}{(1 - r)^{4}} + \frac{P_{5}-F_{c}}{(1 - r)^{5}}$

$ E[x] = - 1.8M + \frac{202192.366(10-8)- 40,000}{(1 - 0.03)^{1}} + \frac{200099.0(10-8)- 40,000}{(1 - 0.03)^{2}} + \frac{0.0(10-8)- 40,000}{(1 - 0.03)^{3}} + \frac{175114.0(10-8)- 40,000}{(1 - 0.03)^{4}} + \frac{49998.4393(10-8)- 40,000}{(1 - 0.03)^{5}} $

$ E[x] = -27,823.85909718645$

Al limitar las distribuciones se observa que la media de la distribución del año 3 es cero, por lo que el método analitico muestra un valor negativo. 

Utilizando la media real de los valores del año tres se observa una esperanza de más cercana al arrojado usando código.

    4) What is the VAR of NPV?
$$ Var[x] = Var[- I] + Var[\frac{Var_{n}-F_{c}}{(1 - r)^{n}}] $$
$$ Var[x] = Var[- 1.8M] + Var[\frac{Var_{1}-F_{c}}{(1 - r)^{1}}] + Var[\frac{Var_{2}-F_{c}}{(1 - r)^{2}}] + Var[\frac{Var_{3}-F_{c}}{(1 - r)^{3}}] + Var[\frac{Var_{4}-F_{c}}{(1 - r)^{4}}] + Var[\frac{Var_{5}-F_{c}}{(1 - r)^{5}}]$$
$$ Var[x] = Var[\frac{99037.349(10-8)- 40,000}{(1 - 0.03)^{1}}] + Var[\frac{299876.0(10-8)- 40,000}{(1 - 0.03)^{2}}] + Var[\frac{400000.0(10-8)- 40,000}{(1 - 0.03)^{3}}] + Var[\frac{149883.0(10-8)- 40,000}{(1 - 0.03)^{4}}] + Var[\frac{99.39(10-8)- 40,000}{(1 - 0.03)^{5}}]$$

$$ Var[x] = 372,841.42601437017 $$

    5) What distribution does the NPV have?
> $\mathcal{N} \sim (-27823.85909718645, 372841.42601437017)$  

    6) What probability of having NPV over a 1 M? 
> 0.0%


### Q2: Maximum Likelihood

Using Maximum Likelihood parameter estimation Derive the process to get the optimal parameter(s): 

Exponential Distribution (get $\hat{\lambda}$):

$$y_{i} \sim \ f(\lambda, y_{i})= \lambda e^{-\lambda y_i}$$


### <span style="background:#FFFF00">Answer 2</span>



### Q3: Modeling Derivatives

Find the differential form of a derivative, $df(X_{t}, t):= df_{t}$, where: (tip: Use Taylor series and Itô Lemma)

$$dX_{t}=\mu_{}X_{t}^3(d_{t}) +X_{t}\sigma(dW_{t}) + \sigma X_{t} \ $$

Is the equation risk fee under neutral valuation? Why?




### <span style="background:red">Answer 3</span>
